In [7]:
import torch

In [8]:
from sr_emu import Generator
from prettytable import PrettyTable

In [9]:
G = Generator(6, 6, 1)

In [10]:
G.cuda()

Generator(
  (convblock0): ConvStyled3d(
    (style_block): Sequential(
      (0): Linear(in_features=1, out_features=6, bias=True)
    )
  )
  (act): LeakyReLUStyled(negative_slope=0.2, inplace=True)
  (addnoise): AddNoise()
  (hblock0): HBlock(
    (act): LeakyReLUStyled(negative_slope=0.2, inplace=True)
    (upsample): Resampler()
    (conv1): ConvStyled3d(
      (style_block): Sequential(
        (0): Linear(in_features=1, out_features=512, bias=True)
      )
    )
    (addnoise1): AddNoise()
    (conv2): ConvStyled3d(
      (style_block): Sequential(
        (0): Linear(in_features=1, out_features=256, bias=True)
      )
    )
    (addnoise2): AddNoise()
    (proj): ConvStyled3d(
      (style_block): Sequential(
        (0): Linear(in_features=1, out_features=256, bias=True)
      )
    )
    (noise_proj1): ConvStyled3d(
      (style_block): Sequential(
        (0): Linear(in_features=1, out_features=6, bias=True)
      )
    )
    (noise_proj2): ConvStyled3d(
      (style_block):

In [11]:
new_state_dict = {
    'model':G.state_dict()
}
torch.save(new_state_dict, '/hildafs/home/xzhangn/xzhangn/emulator_sr/5-training/test_run-2-0/GAN_state/G.pt')


In [12]:
new_G = torch.load('/hildafs/home/xzhangn/xzhangn/emulator_sr/5-training/test_run-2-0/GAN_state/G.pt')
old_G = torch.load('/hildafs/home/xzhangn/xzhangn/emulator_sr/5-training/test_run-2-0/GAN_state/state_710.pt')

In [13]:
new_G_state = new_G['model']
old_G_state = old_G['model']

In [14]:
new_keys = list(new_G_state.keys())
old_keys = list(old_G_state.keys())

In [15]:
print(len(new_keys))
print(len(old_keys))

75
46


In [16]:
freeze_keys = []
for x in new_keys:
    if 'conv' in x:
        freeze_keys.append(x)

In [17]:
print(len(freeze_keys))

28


In [18]:
print(freeze_keys)

['convblock0.weight', 'convblock0.bias', 'convblock0.style_block.0.weight', 'convblock0.style_block.0.bias', 'hblock0.conv1.weight', 'hblock0.conv1.bias', 'hblock0.conv1.style_block.0.weight', 'hblock0.conv1.style_block.0.bias', 'hblock0.conv2.weight', 'hblock0.conv2.bias', 'hblock0.conv2.style_block.0.weight', 'hblock0.conv2.style_block.0.bias', 'hblock1.conv1.weight', 'hblock1.conv1.bias', 'hblock1.conv1.style_block.0.weight', 'hblock1.conv1.style_block.0.bias', 'hblock1.conv2.weight', 'hblock1.conv2.bias', 'hblock1.conv2.style_block.0.weight', 'hblock1.conv2.style_block.0.bias', 'hblock2.conv1.weight', 'hblock2.conv1.bias', 'hblock2.conv1.style_block.0.weight', 'hblock2.conv1.style_block.0.bias', 'hblock2.conv2.weight', 'hblock2.conv2.bias', 'hblock2.conv2.style_block.0.weight', 'hblock2.conv2.style_block.0.bias']


In [19]:
old_keys_conv = []
for x in old_keys:
    if 'std' not in x:
        old_keys_conv.append(x)

In [20]:
print(len(old_keys_conv))
print(old_keys_conv)

40
['block0.0.weight', 'block0.0.bias', 'block0.0.style_block.0.weight', 'block0.0.style_block.0.bias', 'blocks.0.conv.0.weight', 'blocks.0.conv.0.bias', 'blocks.0.conv.0.style_block.0.weight', 'blocks.0.conv.0.style_block.0.bias', 'blocks.0.conv1.0.weight', 'blocks.0.conv1.0.bias', 'blocks.0.conv1.0.style_block.0.weight', 'blocks.0.conv1.0.style_block.0.bias', 'blocks.0.proj.0.weight', 'blocks.0.proj.0.bias', 'blocks.0.proj.0.style_block.0.weight', 'blocks.0.proj.0.style_block.0.bias', 'blocks.1.conv.0.weight', 'blocks.1.conv.0.bias', 'blocks.1.conv.0.style_block.0.weight', 'blocks.1.conv.0.style_block.0.bias', 'blocks.1.conv1.0.weight', 'blocks.1.conv1.0.bias', 'blocks.1.conv1.0.style_block.0.weight', 'blocks.1.conv1.0.style_block.0.bias', 'blocks.1.proj.0.weight', 'blocks.1.proj.0.bias', 'blocks.1.proj.0.style_block.0.weight', 'blocks.1.proj.0.style_block.0.bias', 'blocks.2.conv.0.weight', 'blocks.2.conv.0.bias', 'blocks.2.conv.0.style_block.0.weight', 'blocks.2.conv.0.style_block.0

In [21]:
new_keys_conv = []
for x in new_keys:
    if 'std' not in x and 'noise_proj' not in x:
        new_keys_conv.append(x)
print(len(new_keys_conv))

40


In [22]:
for x, y in zip(new_keys_conv, old_keys_conv):
    new_G_state[x] = old_G_state[y]
new_G['model'] = new_G_state
new_G['pretrained_layers'] = new_keys_conv

In [23]:
torch.save(new_G, '/hildafs/home/xzhangn/xzhangn/emulator_sr/5-training/test_run-2-0/GAN_state/G.pt')

In [24]:
def print_parameters(state):
    table = PrettyTable(["Names", "Parameters", 'Shape'])
    total_params = 0
    for name in state.keys():
        parameter = state[name]
        params = parameter.numel()
        table.add_row([name, params, parameter.shape])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params


In [25]:
print_parameters(new_G['model'])

+------------------------------------------+------------+---------------------------------+
|                  Names                   | Parameters |              Shape              |
+------------------------------------------+------------+---------------------------------+
|            convblock0.weight             |    3072    |  torch.Size([512, 6, 1, 1, 1])  |
|             convblock0.bias              |    512     |        torch.Size([512])        |
|     convblock0.style_block.0.weight      |     6      |        torch.Size([6, 1])       |
|      convblock0.style_block.0.bias       |     6      |         torch.Size([6])         |
|               addnoise.std               |    512     |        torch.Size([512])        |
|           hblock0.conv1.weight           |  3538944   | torch.Size([256, 512, 3, 3, 3]) |
|            hblock0.conv1.bias            |    256     |        torch.Size([256])        |
|    hblock0.conv1.style_block.0.weight    |    512     |       torch.Size([512,

6989426

In [26]:
print_parameters(old_G['model'])

+---------------------------------------+------------+---------------------------------+
|                 Names                 | Parameters |              Shape              |
+---------------------------------------+------------+---------------------------------+
|            block0.0.weight            |    3072    |  torch.Size([512, 6, 1, 1, 1])  |
|             block0.0.bias             |    512     |        torch.Size([512])        |
|     block0.0.style_block.0.weight     |     6      |        torch.Size([6, 1])       |
|      block0.0.style_block.0.bias      |     6      |         torch.Size([6])         |
|     blocks.0.noise_upsample.0.std     |    512     |        torch.Size([512])        |
|         blocks.0.conv.0.weight        |  3538944   | torch.Size([256, 512, 3, 3, 3]) |
|          blocks.0.conv.0.bias         |    256     |        torch.Size([256])        |
|  blocks.0.conv.0.style_block.0.weight |    512     |       torch.Size([512, 1])      |
|   blocks.0.conv.0.s

6979422

In [27]:
input_size = 24
padding = 3
N = input_size + 2 * padding
noise_padding = 4
noise0_N = input_size*2 + 2 * noise_padding
noise1_N = input_size*4 + 2 * noise_padding
noise2_N = input_size*8 + 2 * noise_padding

In [28]:
input = torch.randn(1, 6, N, N, N).cuda()
style = torch.randn(1, 1).cuda()
early_noise = torch.randn(1, 6, N, N, N).cuda()
noise0 = torch.randn(1, 6, noise0_N, noise0_N, noise0_N).cuda()
noise1 = torch.randn(1, 6, noise1_N, noise1_N, noise1_N).cuda()
noise2 = torch.randn(1, 6, noise2_N, noise2_N, noise2_N).cuda()

In [29]:
out = G(input, style, early_noise, noise0, noise1, noise2)

In [30]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   22525 MB |   24090 MB |   43524 MB |   20999 MB |
|       from large pool |   22505 MB |   24070 MB |   43499 MB |   20993 MB |
|       from small pool |      19 MB |      19 MB |      25 MB |       5 MB |
|---------------------------------------------------------------------------|
| Active memory         |   22525 MB |   24090 MB |   43524 MB |   20999 MB |
|       from large pool |   22505 MB |   24070 MB |   43499 MB |

In [31]:
print(torch.cuda.max_memory_allocated())

25260513280


In [32]:
!nvidia-smi

Fri Jun  9 14:04:11 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   27C    P0   104W / 400W |  28118MiB / 40536MiB |     85%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [33]:
loss = out.sum()
loss.backward()

In [34]:
print(torch.cuda.max_memory_allocated())

28110601728


In [36]:
import numpy as np

In [46]:
snap = 99

In [51]:
noise64 = np.load('/hildafs/home/xzhangn/xzhangn/emulator_sr/4-data/2-0/noise/set7/PART_0{}/noise-64.npy'.format(snap))
noise128 = np.load('/hildafs/home/xzhangn/xzhangn/emulator_sr/4-data/2-0/noise/set7/PART_0{}/noise-128.npy'.format(snap))
noise256 = np.load('/hildafs/home/xzhangn/xzhangn/emulator_sr/4-data/2-0/noise/set7/PART_0{}/noise-256.npy'.format(snap))
noise512 = np.load('/hildafs/home/xzhangn/xzhangn/emulator_sr/4-data/2-0/noise/set7/PART_0{}/noise-512.npy'.format(snap))

In [52]:
print(noise64.shape)
print(noise128.shape)
print(noise256.shape)
print(noise512.shape)

(1, 6, 64, 64, 64)
(1, 6, 128, 128, 128)
(1, 6, 256, 256, 256)
(1, 6, 512, 512, 512)


In [50]:
disp = np.load('/hildafs/home/xzhangn/xzhangn/emulator_sr/4-data/2-0/dmo-512/set7/PART_049/disp.npy')
print(disp.shape)

(3, 512, 512, 512)


In [53]:
a = torch.cuda.max_memory_allocated()
print(a)
print(type(a))

28110601728
<class 'int'>


In [54]:
def bytes2gb(bytes_num):
    return bytes_num / 1024 / 1024 / 1024

In [56]:
mem = torch.tensor(bytes2gb(a))
print(mem)
print(type(mem))

tensor(26.1800)
<class 'torch.Tensor'>
